In [0]:
/* 03_gold_gender_breakdown.ipynb
GOAL: Patient count by gender

SOURCE: kardia_silver.silver_patients
OUTPUT: TABLE: kardia_gold.gold_gender_breakdown

TRIGGER: Single batch job with Type-1 MERGE to update latest counts.

NOTE: The Gold Gender Breakdown table uses Delta Liquid Clustering and Predictive Optimization
      to reduce small files and maintain layout efficiency without manual OPTIMIZE or ZORDER. */

In [0]:
%sql
-- 1 ▸ Build or refresh Gold table with pure SQL
    CREATE OR REPLACE TEMP VIEW v_gender_counts AS
    SELECT GENDER,
           COUNT(*) AS patient_cnt
    FROM kardia_silver.silver_patients
    WHERE GENDER IS NOT NULL
    GROUP BY GENDER;

    CREATE TABLE IF NOT EXISTS kardia_gold.gold_gender_breakdown (
    GENDER       STRING,
    patient_cnt  BIGINT
    ) USING DELTA;

    -- Enable Predictive Optimization
    ALTER TABLE kardia_gold.gold_gender_breakdown
    SET TBLPROPERTIES (
      'delta.autoOptimize.optimizeWrite' = 'true',
      'delta.autoOptimize.autoCompact' = 'true'
    );

    MERGE INTO kardia_gold.gold_gender_breakdown AS t
    USING v_gender_counts                        AS s
    ON t.GENDER = s.GENDER
    WHEN MATCHED THEN UPDATE SET *
    WHEN NOT MATCHED THEN INSERT *;

In [0]:
%sql
-- 2. Preview gender distribution
        SELECT GENDER, 
               patient_cnt
        FROM kardia_gold.gold_gender_breakdown
        ORDER BY patient_cnt DESC
